---
comments: true
layout: notebook
title: Class 3- JWT Implimentation
description: Teamteach by Lindsay, Anika, Grace, Samhita
type: collab
courses: { csa: {week: 19} }
---

# JWT Implimentation

## What is a JWT (Java Web Token)?

- Java Web Tokens (JWT) serve as a secure means of transmitting information in JSON format, providing a way to establish trust between different parties in a web application. 

Imagine JWT as a digital envelope that holds essential information securely:

Three Main Components:

Header: Specifies the token type and the signing algorithm, acting as a label on the envelope.
Payload: Encodes significant user details (e.g., user ID, name, roles) equivalent to the letter inside the envelope.
Signature: Created by combining the encoded Header and Payload with a secret key, functioning as a unique stamp to seal the envelope.
Token Integrity Assurance: The Signature ensures the token's integrity; tampering with the Header or Payload breaks the seal, rendering the token invalid.

Stateless Nature of JWT Tokens: These tokens are stateless, implying no server-side storage. Analogous to a school locker key, the server only verifies the validity of the key without storing belongings.

Authorization Mechanism: To restrict token usage to authorized individuals, a secret key is used for signing and verifying authenticity. This process is akin to a secret handshake – knowing it grants access.

Building Blocks Analogy: These JWT concepts serve as foundational blocks facilitating secure communication across different parts of a web application.